In [1]:
import pandas as pd
from datetime import datetime

import numpy as np
from scipy.optimize import minimize

In [2]:
def portfolio_std_dev3(stock_and_qty_dict, start_date, end_date):
    initial_value = 0
    composition_list = []
    
    demeanedreturn = []
    
    for stock,qty in stock_and_qty_dict.items():
        file_string = '/Users/grant/Documents/Stock_Data_Spreadsheets/%s_Data.xlsx'%stock
        data = pd.read_excel(file_string)
        df = pd.DataFrame(data, columns= ['Date','Close'])
        
        df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
        

        initial_value+=df['Close'].iloc[0]*qty
        composition_list.append(df['Close'].iloc[0]*qty)
        
        
        df['Close'] = df['Close'].pct_change()
        mean = df['Close'].mean()
        df['Close']= df['Close']-mean
        demeanedreturn.append(df['Close'].tolist())
    
    demeanedreturn_matrix = np.array(demeanedreturn)
    demeanedreturn_matrix = demeanedreturn_matrix.transpose()
    demeanedreturn_matrix = np.delete(demeanedreturn_matrix,0,axis=0)
    
    covariance_matrix = np.matmul(demeanedreturn_matrix.transpose(),demeanedreturn_matrix)/demeanedreturn_matrix.shape[0]
    
    for i in range(len(composition_list)):
        composition_list[i]/=initial_value
    weights = np.array([composition_list])
    weights = np.transpose(weights)
    
    variance = np.matmul(np.matmul(weights.transpose(),covariance_matrix),weights)
    std_dev = float(variance**0.5)
    
    return std_dev

In [4]:
def portfolio_div(stock_and_qty_dict, start_date, end_date):
    start = datetime.fromisoformat(start_date)
    end = datetime.fromisoformat(end_date)
    total_div = 0

    
    for stock,qty in stock_and_qty_dict.items():
        file_string = '/Users/grant/Documents/Stock_Data_Spreadsheets/%s_Data.xlsx'%stock
        data = pd.read_excel(file_string)
        df = pd.DataFrame(data, columns= ['Date','Div'])
        
        for year in range (start.year,end.year+1):
            modifier = 1
            if year == start.year:
                modifier -= (start.month//3)/4
            if year == end.year:
                modifier -= (4-end.month//3)/4
                
            search_date = '%s-03-01' %year
                
            test = df.loc[df['Date'] == search_date]['Div'].values
            if test.size == 0:
                search_date = '%s-03-04' %year
                test = df.loc[df['Date'] == search_date]['Div'].values
                if test.size == 0:
                    test = [0]
                
            total_div += test[0]*qty*modifier
    return total_div

def portfolio_initial_value(stock_and_qty_dict, start_date):
    initial_value = 0
    for stock,qty in stock_and_qty_dict.items():
        file_string = '/Users/grant/Documents/Stock_Data_Spreadsheets/%s_Data.xlsx'%stock
        data = pd.read_excel(file_string)
        df = pd.DataFrame(data, columns= ['Date','Close'])
        df = df.loc[df['Date']>=start_date]
        initial_value+=df['Close'].iloc[0]*qty
    return initial_value

In [5]:
def optimize_portfolio(stocks,cash,target_yield,start_date,end_date):
    def objective(x):
        my_portfolio={}
        for i in range(len(stocks)):
            my_portfolio[stocks[i]]=x[i]
        
        my_portfolio_std_dev = portfolio_std_dev3(my_portfolio, start_date, end_date)
        return my_portfolio_std_dev

    def constraint1(x):
        my_portfolio={}
        for i in range(len(stocks)):
            my_portfolio[stocks[i]]=x[i]
        
        return cash-portfolio_initial_value(my_portfolio, start_date)

    def constraint2(x):
        my_portfolio={}
        for i in range(len(stocks)):
            my_portfolio[stocks[i]]=x[i]
        
        dividend_yield = portfolio_div(my_portfolio, start_date, end_date)/cash
        return dividend_yield-target_yield
    
    b = (1.0,300.0)
    bnds = (b,b,b)
    con1 = {'type':'eq','fun':constraint1}
    con2 = {'type':'ineq','fun':constraint2}
    cons = [con1,con2]
    x0 = [50,50,50]
    
    sol = minimize(objective,x0,method='SLSQP',bounds=bnds,constraints = cons)
    return sol

In [6]:
optimize_portfolio(["ENB","RY","TD"],10000,0.10,'2018-01-01', '2020-01-01')

     fun: 0.010106903353346566
     jac: array([ 1.66165410e-05, -6.71310117e-05, -5.19220484e-05])
 message: 'Optimization terminated successfully.'
    nfev: 11
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([133.60664619,  17.59828992,  20.00436959])